# This is how our code is designed to run!
## Each loop of reply_to_tweet_bot takes one minute to run
### send_tweet_demo_version has been changed to send_tweet_full_version and now takes a full week before returning a function!

In [ ]:
!pip install billboard.py
!pip install tweepy
import pandas as pd
import numpy as np
import requests
import billboard
import datetime
import tweepy
import time
import logging
import pytz
import dateutil.tz
from pytz import timezone



#this function removes any unwanted chart information from the list and orders the remaining lists to match the key_words list
def narrow_down(anylist):
    final_music_list=[]
    key_words=['The Hot 100','Billboard 200','Pop Songs','Hot Rap Songs','Hot Country Songs','Hot Rock Songs','Alternative Songs','Hot R&B Songs',
              'Hot Latin Songs','Hot Dance/Electronic Songs', 'Korea K-Pop Hot 100','Smooth Jazz Songs','Hot Gospel Songs']
#takes keywords list and if that dictionary exists, it'll move that dictionary to a new, narrowed down list
    for dictionary in anylist:
        for name in key_words:
            if dictionary['Chart Name'] == name:
                final_music_list.append(dictionary)
                break
    return final_music_list
    
    
    
#function to seperate the featuring artist from the main artist, add featuring artist into dictionaries in music_list
def iso_feature_artists(listofcharts):
    middle_music_list = listofcharts.copy() 
#this line runs through every value under the 'Number One Artist' key
    top_artist_incl_fts=[artist['Number One Artist'] for artist in middle_music_list]
    for element in middle_music_list:
#if the word featuring appears in the artists' name, it splits at artist and assigns it to the featuring artist key
        if ' Featuring' in element.get('Number One Artist'):
            artists = element.get('Number One Artist').split(' Featuring ')
            element['Number One Artist']=artists[0]
            element['Featuring Artist']=artists[1]
#if 'featuring' does not appear, 'Featuring Artist' key is assigned to 'No Featuring Artist'value
        else: 
            element['Featuring Artist'] = 'No Featuring Artist'       
    return middle_music_list
    
    
    
#Organizing chart information into a list of tweetable strings
def tweet_text(nearly_final_list):
    tweet_list=[]
    for chart in nearly_final_list:
        tweet_ready_text=('Billboard Chart Name: '+chart['Chart Name']+'\nNumber One Song: '+chart['Number One Song']+
        '\nNumber One Song\'s Artist: '+chart['Number One Artist']+'\nFeaturing Artist(s): '+chart['Featuring Artist']+
        '\nPosition on Last Week\'s Chart: '+ str(chart['Last Week\'s Song Position'])+'\nConsecutive Weeks on Chart: '+ str(chart['Consecutive Weeks on Chart'])+'\n\n\n')
        tweet_list.append(tweet_ready_text)
    return tweet_list
    
    
#this function tweets out the tweets assembled in tweet_text with a delay between each tweet
#the delay is constructed using loops that act as bots and will respond to our mentions on twitter if the mention is formatted correctly
def send_tweet_full_version (list_of_tweets):
    even_or_odd_count = 2
    tweet_count = 0
    text_count = tweet_count + 1
    for tweet in list_of_tweets:
#On the last day (Sunday), the last chart will print and this condition will return True
#when it does, there will be almost a full day's delay and the return value will break out of the function
        if even_or_odd_count == len(list_of_tweets)+1:
            my_api.update_status(status = list_of_tweets[tweet_count])
            print(f'All {text_count} have been published!')
            time.sleep(5)
            print('That\'s the end of the week! Looking forward to next week\'s charts!')
            reply_to_tweet_bot(my_api,tweet_reply_keywords,1422,dont_tweet_list) 
            return ' '
#the thrid argument in reply_to_tweet_bot is now how many minutes the code will loop for
#we use the % function to distinguish between the first and second tweet of the day so we can tweet 
#once at 12PM and once at 5PM per day
        elif even_or_odd_count % 2 == 0:
            my_api.update_status(status = list_of_tweets[tweet_count]) 
            print(f'{text_count} tweets have been sent out!')
            tweet_count+=1
            even_or_odd_count+=1
            text_count+=1
            reply_to_tweet_bot(my_api,tweet_reply_keywords,300,dont_tweet_list)
        elif even_or_odd_count % 2 == 1:
            print(f'{text_count} tweets have been sent out!')
            my_api.update_status(status = list_of_tweets[tweet_count])
            tweet_count+=1
            even_or_odd_count+=1
            text_count+=1
            reply_to_tweet_bot(my_api,tweet_reply_keywords,1140,dont_tweet_list) 
            
            
#this function gathers chart data based on the user's desired input
def tweet_user_request (chart_req,date_req):
#this chunk creats a dictionary and creates keys and matching values in it based on the user's input
    request_dict ={}
    request_dict['Chart Name'] = billboard.ChartData(chart_req,date_req,timeout=None).title
    request_dict['Number One Song']=billboard.ChartData(chart_req,date_req,timeout=None)[0].title
    request_dict['Number One Artist'] = billboard.ChartData(chart_req,date_req,timeout=None)[0].artist
    request_dict['Featuring Artist'] = ' '
#this splits featuring and with, just like in (iso_feature_artists)
    if ' Featuring' in request_dict['Number One Artist']:
        artists = request_dict['Number One Artist'].split(' Featuring ')
        request_dict['Number One Artist']=artists[0]
        request_dict['Featuring Artist']=artists[1]
    elif ' With ' in request_dict['Number One Artist']:
        artists = request_dict['Number One Artist'].split(' With ')
        request_dict['Number One Artist']=artists[0]
        request_dict['Featuring Artist']=artists[1]
    else: 
        request_dict['Featuring Artist'] = 'No Featuring Artist'
    logging.info('Your tweet has been assembled!')
    return request_dict


#formats text so it will be accepted in billboard's syntax
def create_inputable_chart_data (chart_req):
#Despite the word 'top' appearing in chart list names when printed, the billboard syntax does not include it in its chart input syntax
#This removes the words 'top' if it starts the string
    chart_req_first_three_letters = chart_req[0:3]
    if chart_req_first_three_letters == 'top':
        chart_req = chart_req[4:]
#r&b charts are formatted as 'r-and-b' so this code converts &'s to and's
    if '&' in chart_req:
        chart_req_no_and = chart_req.split('&amp;')
        chart_req = chart_req_no_and[0]+'-and-'+chart_req_no_and[1]
    print(chart_req)
    return chart_req


#function acts as a reply bot - it scrapes our mentions and outputs chart info based on the user's request
#every path through this takes about 60 seconds
def reply_to_tweet_bot (api,keywords,loop_count,dont_tweet_list):
    since_id = 1
    check_count = 0
    logging.root.setLevel(logging.INFO)
    #while loop essentially acts as a timer-when bot isn't tweeting one of the 13 charts, it's looking
    #to see if anyone requested a chart. The function runs x amount of times and each loop takes sixty seconds.
    #This enables us to control when our 13 chart tweets will be sent out.
    while True:
            try:
                #This block reads through our account's mentions 
                new_since_id = since_id
                logging.info("Looking through your mentions...")
                for tweet in tweepy.Cursor(api.mentions_timeline, since_id=since_id).items():
                    new_since_id = max(tweet.id, new_since_id)
                #this if statement prevents us from replying to the same tweet twice
                    if any(ids in tweet.id_str for ids in dont_tweet_list):
                        logging.info('All tweets have been successfully answered!\nI\'ll try again in a minute!')
                        time.sleep(60)
                        check_count+=1
                        if check_count == loop_count:
                            return dont_tweet_list
                    else:
                        if tweet.in_reply_to_status_id is not None:
                            continue
                #this block triggers a reply to a tweet if it includes all of our keywords
                        elif all(keyword in tweet.text.lower() for keyword in keywords):
                #changes time to EST as opposed to GMT+/- 0
                            kind_of_persons_at = {tweet.user.screen_name}
                            tz = pytz.timezone('America/New_York')
                            ct = datetime.datetime.now(tz=tz)
                            local_time=ct.strftime("%-I:%M %p")
                #persons_at splits the returned tweet info to retrieve the user's handle
                            persons_at = str(kind_of_persons_at).split("'")[1]
                #this chunk replies to the tweet
                            reply_tweet = api.update_status(f'@{persons_at} Your message was received at {local_time} Eastern Standard time!'
                                                            f' Check back soon for your published Billboard data!',tweet.id)
                            logging.info(f'{tweet.user.name} mentioned you in a tweet!')
                            time.sleep(5)
                            logging.info(f"Responding to {tweet.user.name}...")
                            time.sleep(5)
                            reply_tweet
                            
                        #below try/except gathers user's request and formats it appropriately. If an Index error occurs,
                        #it means the user did not follow the correct template, which is caught in the except net 
                            try:
                                req_tweet_text = tweet.text
                                split_at_colon=str(req_tweet_text).split(': ',2)
                                tweetable_chart_req = split_at_colon[1].splitlines()[0]
                                almost_chart_req = tweetable_chart_req.lower()
                                almost_chart_req = almost_chart_req.split()
                                chart_req = '-'.join(almost_chart_req)
                                user_middle_req_date = split_at_colon[2].splitlines()[0]
                                user_req_date = user_middle_req_date.split('-',2)
                                date_req = user_req_date[2]+'-'+user_req_date[0]+'-'+user_req_date[1]
                                chart_req = create_inputable_chart_data(chart_req)
                                tweetable_chart_req=tweetable_chart_req.title()
                                #this tries tweeting out the expected return to a user's request.
                                #If something goes wrong, the below except nets will catch it.
                                try:
                                    request_dictionary = tweet_user_request(chart_req,date_req)
                                    date_req = datetime.datetime.strptime(date_req,'%Y-%m-%d')
                                    date_req = date_req.strftime("%B %d, %Y")
                                #now that we have all our values, we can print and format the tweet 
                                    if 'Amp;' in tweetable_chart_req:
                                        almost_chart_req=tweetable_chart_req.split('Amp;')
                                        tweetable_chart_req=''.join(almost_chart_req)
                                    my_api.update_status(status = f'Per @{persons_at}\'s request, we\'re publishing the top song from Billboard\'s'
                                                                f' {tweetable_chart_req} chart from {date_req}' 
                                                                 f'\n\nNumber One Song: {request_dictionary["Number One Song"]}'
                                                                  f'\nNumber One Song\'s Artist: {request_dictionary["Number One Artist"]}'
                                                                   f'\nFeaturing Artist(s): {request_dictionary["Featuring Artist"]}'
                                                                                           )
                                    logging.info('The chart has been published! Check out @ChartBotProject to see the results!')
                                    time.sleep(45)
                                    check_count+=1
                                    
                                 #this net catches a tweet if either chart_req or dat_req are incompatible values
                                except (billboard.BillboardNotFoundException, ValueError):
                                    second_reply = api.update_status(f'Sorry {tweet.user.name}!\n\nUnfortunately we couldn\'t'
                                                f' produce the chart you requested. Please check your formatting and'
                                                 f' re-submit the request. Request completed at {local_time}',reply_tweet.id)
                                    second_reply
                                    logging.info(f'A reply has been sent! Hopefully {tweet.user.name} fixes their request!')
                                    time.sleep(45)
                                    check_count+=1
                                  
                                #this except net catches a tweet if the same user had previously requested the same chart
                                except tweepy.error.TweepError:
                                    already_published_reply = api.updtate_status(f'Sorry {tweet.user.name}!\n\nIt appears as if you'
                                                               f'have requested this chart before.'
                                                               f'\\nIf you\'re still curious about the chart leader, try scrolling up'
                                                               f' in your mentions! Request completed at {local_time}',reply_tweet.id)
                                    logging.info('This tweet already exists!! Looking for new requests...')
                                    already_published_reply
                                    time.sleep(45)
                                    check_count+=1
                                    
                        #this except error catches improper formatting in the request and replies to the user's 
                        #tweet notifying them of this error. It asks the user to please reformat their request and try again 
                            except IndexError:
                                logging.info(f'{tweet.user.name}\'s request was formatted incorrectly!')
                                second_reply = api.update_status(f'Sorry {tweet.user.name}!\n\nUnfortunately we couldn\'t'
                                                f' produce the chart you requested. Please check your formatting and'
                                                f' re-submit the request. Request completed at {local_time}',reply_tweet.id)
                                second_reply
                                logging.info(f'A reply has been sent! Hopefully {tweet.user.name} fixes their request!')
                                time.sleep(5)
                                check_count+=1
                            #this if statement adds the tweet id to a list of tweet ids to ensure we don't respond to the same tweet twice
                            if reply_tweet.in_reply_to_status_id_str == tweet.id_str:
                                dont_tweet_list.append(str(reply_tweet.in_reply_to_status_id_str))
                                print(dont_tweet_list)
                                logging.info(f'{tweet.user.name} has been answered!')
                                logging.info(f'Searching for more users to reply to...')
                                if check_count == loop_count:
                                    return dont_tweet_list
                                time.sleep(3)
                            #this else statement is triggered if no tweet was responded to
                            else: 
                                logging.info('All tweets have been successfully answered!\nI\'ll try again in a minute!')
                                check_count+=1
                                if check_count == loop_count:
                                    return dont_tweet_list
            #this exception occurs if the same tweet is sent out twice. This catches a user trying to flood our mentions
            except tweepy.TweepError:
                logging.info('No new tweets!\nI\'ll try again in a minute!')
                time.sleep(60)
                check_count+=1
                if check_count == loop_count:
                    return dont_tweet_list 
                


In [ ]:
week_count = 1
while True: 
#this condition breaks the code if its been running for a year
    if week_count == 52:
        break
    else:
    today = datetime.date.today()
    last_monday = today + dateutil.relativedelta.relativedelta(weekday=dateutil.relativedelta.MO(-1))
    this_weeks_date = last_monday.strftime("%B %d, %Y")
    print(f'\n\n\nThis is Week {week_count} of this bot\'s lovely exitence.'
          f'\nPrinting Billboard Charts for the week of {this_weeks_date}...\n\n')
    music_list = []
    count=0
    try:
#this retrieves information from the number one song from every billboard chart and adds it to a list of dictionaries
        for title in billboard.charts():
            music_dict = {}
            title_every_list = billboard.ChartData(title, timeout=None).title
            music_dict['Chart Name'] = title_every_list
            top_song = billboard.ChartData(title, timeout=None)[0].title
            music_dict['Number One Song'] = top_song
            top_artist = billboard.ChartData(title, timeout=None)[0].artist
            music_dict['Number One Artist'] = top_artist
            top_song_img = billboard.ChartData(title, timeout=None)[0].image
            music_dict['Song Image URL'] = top_song_img
            last_song_pos = billboard.ChartData(title, timeout=None)[0].lastPos
            music_dict['Last Week\'s Song Position'] = last_song_pos
            consec_weeks = billboard.ChartData(title, timeout=None)[0].weeks
            music_dict['Consecutive Weeks on Chart'] = consec_weeks
            music_dict['Featuring Artist'] = ' '
            music_list.append(music_dict)
            count+=1
            print(str(count)+'/168 charts loaded!')
        print('All done! All charts are loaded!\nTweeting in progress, hang tight...')
#these functions turn the retrieved info into well-formatted strings that can be tweeted
        narrowed_down_list = narrow_down(music_list)
        ready_to_tweet_list = iso_feature_artists(narrowed_down_list)
        tweet_strings = tweet_text(ready_to_tweet_list)
#this is how tweepy knows to access my perticular twitter account
    consumer_token = 'N4S1gmpEfkPtYX0dWn0Y1ke0n'
    consumer_token_secret = 'PhFe83V3fN0aOjP3WYufV1BjatlErkjmBP21g1ipNuDEaH20gw'
    my_access_token = '1256378168644112393-ngpkFIfK4A07afvyTRkmc50rYZskMs'
    my_access_token_secret = 'xEI5tSFPwkq0GLddKJIUCbVU7KrDonRxUAsGIsaRGgc9f'
    my_auth = tweepy.OAuthHandler(consumer_token, consumer_token_secret)
    my_auth.set_access_token(my_access_token, my_access_token_secret) 
    my_api = tweepy.API(my_auth)
#twitter searches tweets in our mentions with these keywords and only replies if the tweet includes all of them
    tweet_reply_keywords = ['chart',':','-','date']
#this list contains the tweet id for our pinned tweet and makes it exempt from being replied to
    dont_tweet_list = ['1257128566145179649'] 
    send_tweet_full_version(tweet_strings)
    week_count+=1
#this exception catches if the account tries to tweet charts that have already been tweeted-
#this will only happen if 
except tweepy.error.TweepError:
    print('The new charts haven\'t been pulished yet!\n\nRetrying in 5 minutes!')
    time.sleep(1800)
    